In [48]:
import MDAnalysis
import MDAnalysis.transformations
import requests
from os import mkdir, path
from pathlib import Path
from matplotlib import pyplot as plt
import imageio
from IPython.display import Video
import nglview
import numpy

In [2]:
PDB_DIR = Path("./pdb/")

In [3]:
def download_pdb(pdb_id: str):
    if not path.isdir(PDB_DIR):
        mkdir(PDB_DIR)
    download_path = PDB_DIR / f"{pdb_id}.pdb"
    source_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    
    with requests.get(source_url, stream=True) as response:
        response.raise_for_status()
        with open(download_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192): 
                f.write(chunk)

    return download_path

In [19]:
filename = download_pdb("4UV3")

In [43]:
universe = MDAnalysis.Universe(filename)
pore = universe.select_atoms("chainid A B C D E F G H I")

view = nglview.show_mdanalysis(pore)
view.clear_representations()
view.add_cartoon(opacity=0.2)
view.add_licorice(selection="51-56")

view

NGLWidget()

In [20]:
help(nglview.show_mdanalysis)

Help on function show_mdanalysis in module nglview.show:

show_mdanalysis(atomgroup, **kwargs)
    Show NGL widget with MDAnalysis AtomGroup.
    
    Can take either a Universe or AtomGroup as its data input.
    
    Examples
    --------
    >>> import nglview as nv # doctest: +SKIP
    ... import MDAnalysis as mda
    ... u = mda.Universe(nv.datafiles.GRO, nv.datafiles.XTC)
    ... prot = u.select_atoms('protein')
    ... w = nv.show_mdanalysis(prot)
    ... w



In [108]:
universe = MDAnalysis.Universe(filename)
with imageio.get_writer('test.mp4', fps=20) as video:

    for _ in range(360):
        MDAnalysis.transformations.rotate.rotateby(1, direction=[1, 0, 0], ag=universe.atoms)(universe.trajectory.ts)
        plt.figure(figsize=[5.12, 5.12])
        plt.xlim(-100, 100)
        plt.ylim(-50, 150)
        for chain in "ABCDEFGHI":
            selection = universe.select_atoms(f"backbone and chainid {chain}")
            plt.plot(selection.positions[:, 1], selection.positions[:, 0])

        plt.savefig("temp.png")
        video.append_data(imageio.v2.imread("temp.png"))
        plt.close()

Video("test.mp4")

In [107]:
universe = MDAnalysis.Universe(filename).select_atoms("chainid A B C D E F G H I")
positions = universe.atoms.positions

slice_centre = 41
slice_thickness = 4


with imageio.get_writer('slices.mp4', fps=20) as video:

    for slice_centre in numpy.linspace(-20, 120, 240):
        behind = positions[(positions[:, 2] < slice_centre + slice_thickness)]
        sliced = positions[(positions[:, 2] < slice_centre + slice_thickness) & (positions[:, 2] > slice_centre - slice_thickness)]
        
        plt.figure(figsize=[5.12, 5.12])
        plt.xlim(-30, 110)
        plt.ylim(-60, 10)

        plt.scatter(behind[:,0], behind[:,1], color="cyan", alpha=0.05)
        plt.scatter(sliced[:,0], sliced[:,1], color="blue", alpha=0.1)

        plt.savefig("temp.png")
        video.append_data(imageio.v2.imread("temp.png"))
        plt.close()

Video("slices.mp4")

In [94]:
numpy.linspace(0, 10, 3)

array([ 0.,  5., 10.])